In [1]:
import numpy as np

In [8]:
import pandas as pd
data = pd.read_csv("results/metrics_regression.csv")
data.head(10)

,dataset,task_type,model,metric,value,scenario
0,toy,regression,RandomForest,MSE,1.881405,scenario1
1,toy,regression,RLT,MSE,14.804511,scenario1


on génère un problème de classification binaire où les variables explicatives sont indépendantes et uniformes entre 0 et 1.
La probabilité que Y=1 dépend de manière non-linéaire des deux premières variables, et on utilise la fonction de répartition normale pour transformer cette combinaison en une probabilité entre 0 et 1.
enfin, on génère la classe Y à partir d’une loi Bernoulli.
Ce scénario permet de tester des méthodes de classification dans un cadre simple mais non-linéaire.
X[:,0] = 𝑋 ( 1 )
X[:,1] = 𝑋 ( 2 ) 
norm.cdf() = fonction Φ (CDF normale standard)
Pourquoi utiliser une CDF normale ?
Pour obtenir des probabilités dans [0,1] à partir d’une combinaison linéaire.

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# Parameters
N = 100
p = 2  # at least 2 variables required

# Generate X ~ Unif[0,1]^p
X = np.random.uniform(0, 1, size=(N, p))

# Compute mu_i = Φ(10 * (X1 - 1) + 20 * |X2 - 0.5|)
mu = norm.cdf(10 * (X[:,0] - 1) + 20 * np.abs(X[:,1] - 0.5))

# Generate Y_i ~ Bernoulli(mu_i)
Y = np.random.binomial(1, mu)

# Build DataFrame
df = pd.DataFrame({
    "X1": X[:,0],
    "X2": X[:,1],
    "mu": mu,
    "Y": Y
})

# Save to CSV
df.to_csv("dataset_scenario1.csv", index=False)

print("Dataset saved as dataset_scenario1.csv")
print(df.head())


Dataset saved as dataset_scenario1.csv
         X1        X2            mu  Y
0  0.705134  0.702707  8.655238e-01  1
1  0.245582  0.370468  3.643706e-07  0
2  0.684008  0.572693  4.399819e-02  0
3  0.240366  0.659481  5.247460e-06  0
4  0.150357  0.728866  4.443904e-05  0


on génère un modèle de régression non linéaire où les variables explicatives sont indépendantes et uniformes entre 0 et 1.
La réponse dépend uniquement des deux premières variables, à travers des termes tronqués (parties positives), ce qui crée une relation active seulement lorsque 𝑋 ( 1 ) > 0.5 X (1) >0.5 et 𝑋 ( 2 ) > 0.25 X (2) >0.25.
Cette structure introduit une non-linéarité forte, car la fonction s’annule pour une grande partie de l’espace des covariables, puis augmente rapidement ailleurs.
enfin, on ajoute un bruit gaussien pour simuler une réponse réelle.
np.maximum(a,0) = ( 𝑎 ) + 
(a) + = partie positive 
pourquoi la partie positive ? → Pour créer une fonction non linéaire qui s’active seulement au-delà d’un certain seuil.

In [3]:
import numpy as np
import pandas as pd

# Parameters
N = 100      # number of samples
p = 2        # dimensionality (we need at least 2)

# Generate X ~ Unif[0,1]^p
X = np.random.uniform(0, 1, size=(N, p))

# Noise
epsilon = np.random.normal(0, 1, size=N)

# Compute Y according to the model
# (x)^+ means max(x,0)
Y = 100 * np.maximum(X[:,0] - 0.5, 0)**2 * np.maximum(X[:,1] - 0.25, 0) + epsilon

# Build DataFrame
df = pd.DataFrame({
    "X1": X[:,0],
    "X2": X[:,1],
    "Y": Y
})

# Save to CSV
df.to_csv("dataset_scenario2.csv", index=False)

print("Dataset saved as dataset_scenario2.csv")
print(df.head())


Dataset saved as dataset_scenario2.csv
         X1        X2         Y
0  0.319322  0.951037  1.148355
1  0.755862  0.344935 -0.217128
2  0.230846  0.424519  0.496688
3  0.427327  0.472000  1.590026
4  0.385562  0.995382  0.800516


In [4]:
import numpy as np
import pandas as pd

# Parameters
N = 300
p = 200  # must be >= 200

# Build covariance matrix Σ_ij = 0.9^{|i-j|}
rho = 0.9
indices = np.arange(p)
Sigma = rho ** np.abs(indices.reshape(-1, 1) - indices.reshape(1, -1))

# Generate X ~ N(0, Σ)
X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma, size=N)

# Noise
epsilon = np.random.normal(0, 1, size=N)

# Compute Y = 2 X50 X100 + 2 X150 X200 + noise
Y = (
    2 * X[:, 49] * X[:, 99] +   # X^(50) * X^(100)
    2 * X[:, 149] * X[:, 199] + # X^(150) * X^(200)
    epsilon
)

# Build DataFrame with X1,...,Xp
df = pd.DataFrame(X, columns=[f"X{i+1}" for i in range(p)])
df["Y"] = Y

# Save to CSV
df.to_csv("dataset_scenario3.csv", index=False)

print("Dataset saved as dataset_scenario3.csv")
print(df.head())


Dataset saved as dataset_scenario3.csv
         X1        X2        X3        X4        X5        X6        X7  \
0 -0.688638 -1.066375 -0.460146 -0.059467 -0.271955 -0.131493 -0.788809   
1 -0.411813  0.034179 -0.227969  0.325822 -0.036876 -0.413824 -0.498281   
2  1.008238  0.459569  0.491861  0.453412  0.755115 -0.096177 -0.658120   
3 -1.746562 -1.327608 -1.011336 -0.806175 -1.226004 -0.725211 -0.701643   
4 -0.604899 -0.530167 -0.014808  0.324090 -0.093750 -0.582721 -0.062468   

         X8        X9       X10  ...      X192      X193      X194      X195  \
0 -0.737242 -1.156370 -1.482951  ...  0.450268 -0.098392  0.613270  0.891197   
1 -1.501095 -1.305745 -1.319844  ...  0.039779 -0.119592 -0.389605 -0.519751   
2 -0.480647  0.068644  0.200754  ...  1.318274  0.917366  0.661210  1.119629   
3 -0.712942 -0.444518 -0.075309  ...  2.288081  2.320963  1.992203  2.211537   
4  0.206996  0.427822  0.499930  ... -0.171055  0.393061  0.880157  0.091806   

       X196      X197      X1

In [5]:
import numpy as np
import pandas as pd

# Parameters
N = 200
p = 150  # must be >= 150

# Build covariance matrix Σ_ij = 0.5^{|i-j|} + 0.2 * I(i ≠ j)
indices = np.arange(p)
Sigma = 0.5 ** np.abs(indices.reshape(-1, 1) - indices.reshape(1, -1))
Sigma += 0.2 * (1 - np.eye(p))

# Generate X ~ N(0, Σ)
X = np.random.multivariate_normal(mean=np.zeros(p), cov=Sigma, size=N)

# Noise
epsilon = np.random.normal(0, 1, size=N)

# Compute Y = 2 X50 + 2 X100 + 4 X150 + noise
Y = 2*X[:, 49] + 2*X[:, 99] + 4*X[:, 149] + epsilon

# Build DataFrame with X1,...,Xp
df = pd.DataFrame(X, columns=[f"X{i+1}" for i in range(p)])
df["Y"] = Y

# Save to CSV
df.to_csv("dataset_scenario4.csv", index=False)

print("Dataset saved as dataset_scenario4.csv")
print(df.head())


Dataset saved as dataset_scenario4.csv
         X1        X2        X3        X4        X5        X6        X7  \
0 -0.251218  0.753262 -0.292834 -1.207095 -0.700615 -1.254435 -0.200197   
1  0.118997  1.192550  1.390108 -0.227345 -1.820100 -1.340123 -0.797827   
2  1.766989  1.413203  1.779663  1.096476  1.666644  1.557943  0.920361   
3 -0.555432 -0.382622  1.116322  1.436634  0.631923 -0.904914  0.429126   
4 -0.106918 -0.710725 -0.667337 -0.719477 -0.486179 -0.771772 -0.545252   

         X8        X9       X10  ...      X142      X143      X144      X145  \
0 -0.278388 -1.015858  0.309904  ... -1.422174 -2.296872 -2.104639 -2.363439   
1 -0.390845 -0.101781 -0.248071  ...  0.725688  0.386470  0.080163  0.723598   
2  1.434563  0.402589  0.177028  ... -0.445086 -0.401321 -0.250543 -0.003438   
3 -0.260528 -1.693982 -1.811768  ... -0.047541  1.021219  0.463198  1.058450   
4 -0.402469 -0.430549  0.013423  ...  0.231953 -0.233694 -0.078047  0.643347   

       X146      X147      X1

In [6]:
import pandas as pd

# Chemin vers ton fichier dans ton ordinateur
input_path = r"C:\Users\bahae\Downloads\ozone.txt"
output_path = r"C:\Users\bahae\Downloads\ozone.csv"

# Lire le fichier .txt séparé par des ;
df = pd.read_csv(input_path, sep=";", encoding="utf-8")

# Sauvegarder en CSV
df.to_csv(output_path, index=False)

print("Conversion terminée ! Le fichier ozone.csv est créé dans ton dossier Downloads.")


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\bahae\\Downloads\\ozone.txt'

In [ ]:
import pandas as pd

input_path = r"D:\archive (1)\parkinsons.data"
output_path = r"D:\parkinsons.csv"

df = pd.read_csv(input_path, sep=",", header=None)

# Ajouter les noms de colonnes (officiel du dataset)
df.columns = [
    "name", "MDVP:Fo(Hz)", "MDVP:Fhi(Hz)", "MDVP:Flo(Hz)",
    "MDVP:Jitter(%)", "MDVP:Jitter(Abs)", "MDVP:RAP", "MDVP:PPQ",
    "Jitter:DDP", "MDVP:Shimmer", "MDVP:Shimmer(dB)", "Shimmer:APQ3",
    "Shimmer:APQ5", "MDVP:APQ", "Shimmer:DDA", "NHR", "HNR",
    "RPDE", "DFA", "spread1", "spread2", "D2", "PPE", "status"
]

df.to_csv(output_path, index=False)

print("✔ Conversion done ! File saved as parkinsons.csv")


In [ ]:
import pandas as pd

# Charger ton fichier CSV qui contient la ligne dupliquée
df = pd.read_csv("parkinsons.csv")

# Supprimer la LIGNE qui a dupliqué les colonnes
df = df.drop(index=0)  # si la ligne dupliquée est la première
# df = df.drop(index=1)  # sinon, si elle est la deuxième

# Réinitialiser l’index proprement
df = df.reset_index(drop=True)

# Sauvegarder un CSV propre
df.to_csv("parkinsons_clean.csv", index=False)

print("✔ Fichier corrigé : parkinsons_clean.csv")
